In [46]:
import torch
from torch import nn as nn
from torch.nn import functional as F
import numpy as np
from tqdm import tqdm 
import wandb
import numpy as np
import os

In [ ]:
# For Google Colab run

# from google.colab import drive
# drive.mount('/content/drive')

# !pip install wandb
# !wandb login
# content = "/content/drive/MyDrive/content.txt"

In [57]:
# Device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
# Hyperparameters
block_size = 16
batch_size = 8
max_iterations = 20
lr = 3e-4
eval_iterations = 100
dropout = 0.2
num_embd = 384 # Total dimensions to capture in the embedding
num_layers = 8
num_head = 8
split_factor = 0.9

In [42]:
content = "content.txt" if 'content' not in globals() else globals()['content']
with open(content, 'r', encoding='utf-8') as f:
    for line in f:
        text = f.read()

split = int(len(text) * split_factor) 
train_data = text[:split]
test_data = text[split:]

In [50]:
# --- GPT-SW3-TOKENIZER --- 

# Try to use string level tokenizer
from transformers import AutoTokenizer
import sentencepiece

model_name = "AI-Sweden-Models/gpt-sw3-126m"
# Import AI Sweden's tokenizer 
tokenizer = AutoTokenizer.from_pretrained(model_name)

encode = tokenizer.encode
decode = tokenizer.decode

# Vocabulary size 
vocab_size = tokenizer.vocab_size

# Encode
train_ids = encode(train_data)
val_ids = encode(test_data)
# print(f"Train data has {len(train_ids)} tokens")
# print(f"Val data has {len(val_ids)} tokens")

train_ids = np.array(train_ids, dtype=np.uint16)
val_ids = np.array(val_ids, dtype=np.uint16)

train_ids.tofile(os.path.join(os.getcwd(), 'train.bin'))
val_ids.tofile(os.path.join(os.getcwd(), 'val.bin'))

Token indices sequence length is longer than the specified maximum sequence length for this model (504295 > 2048). Running this sequence through the model will result in indexing errors


In [26]:
## --- CHAR LEVEL TOKENIZER --- ##
characters = sorted(list(set(text)))
vocab_size = len(characters)

# Creating a mapping from unique characters to indices
string_to_int = { char:index for index, char in enumerate(characters)}
int_to_string = { index:char for index, char in enumerate(characters)}

# Encoding the text
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda c : ''.join([int_to_string[i] for i in c])

# Turn into tensors 
data = torch.tensor(encode(text), dtype=torch.long)

In [68]:
train_data = np.memmap('train.bin', dtype=np.uint16, mode='r')
test_data = np.memmap('val.bin', dtype=np.uint16, mode='r')

# train_data = data[:split].to(device)
# test_data = data[split:].to(device)

def get_batch(split: str):
    """Returns a random batch from the data"""
    data = train_data if split == "train" else test_data
    start_index = torch.randint(len(data) - block_size, (batch_size,))
    # print(start_index)
    x = torch.stack([torch.from_numpy((data[idx:idx+block_size]).astype(np.int64)) for idx in start_index])
    y = torch.stack([torch.from_numpy((data[idx+1:idx+block_size+1]).astype(np.int64)) for idx in start_index])
    # x = torch.stack([data[idx:idx+block_size] for idx in start_index])
    # y = torch.stack([data[idx+1:idx+block_size+1] for idx in start_index])
    x, y = x.to(device), y.to(device) # Make sure data is on the right device
    return x, y

In [52]:
def estimate_loss(model: nn.Module):
    out = {}
    model.eval()
    with torch.inference_mode():
        for split in ['train', 'val']:
            losses = torch.zeros(eval_iterations)
            for k in range(eval_iterations):
                X, Y = get_batch(split)
                logits, loss = model(X, Y)
                losses[k] = loss.item()
            out[split] = losses.mean()
    model.train()
    return out 

In [53]:
class Head(nn.Module):
    """One head of self-attention"""
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(num_embd, head_size, bias=False)
        self.query = nn.Linear(num_embd, head_size, bias=False)
        self.value = nn.Linear(num_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Input shape (Batch, time-step, channels)
        # Output shape: (Batch, time-step, head size)
        B, T, C = x.shape
        # Calling the transformations, but different ones for each
        k = self.key(x) # (B, T, head size)
        q = self.query(x) # (B, T, head size)
        # Compute attention scores
        weights = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        weights = weights.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # Mask out upper triangular 
        # Apply softmax along the last dimension 
        weights = F.softmax(weights, dim=-1) # (B, T, T)
        weights = self.dropout(weights)
        # Apply attention
        v = self.value(x) # (B, T, head size)
        out = weights @ v # (B, T, head size)
        return out

In [54]:
class MultiHeadAttention(nn.Module):
    """ Multiple heads of self-attention in parallel """
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(in_features=head_size * num_heads, out_features=num_embd)
        self.dropout = nn.Dropout(dropout) 
    
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) where F = head_size * num_heads
        out = self.dropout(self.proj(out)) # (B, T, E)
        return out

In [55]:
class FeedForward(nn.Module):
    """A simple feed forward with linear layers and ReLU activations"""
    def __init__(self, num_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_features=num_embd, out_features=4*num_embd),
            nn.ReLU(),
            nn.Linear(in_features=4*num_embd, out_features=num_embd),
            nn.Dropout(dropout) # Dropout to prevent overfitting
        )
    
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """Implementation of a Transformer block"""
    def __init__(self, num_embd, num_head):
        super().__init__()
        head_size = num_embd // num_head 
        self.self_attention = MultiHeadAttention(num_head, head_size)
        self.feed_forward = FeedForward(num_embd)
        self.ln1 = nn.LayerNorm(num_embd)
        self.ln2 = nn.LayerNorm(num_embd)

    def forward(self, x):
        y = self.self_attention(x)
        x = self.ln1(x + y)
        y = self.feed_forward(x)
        x = self.ln2(x + y)
        return x


class GPTModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, num_embd)
        self.position_embedding_table = nn.Embedding(block_size, num_embd)
        self.blocks = nn.Sequential(*[Block(num_embd, num_head=num_head) for _ in range(num_layers)])
        self.ln_f = nn.LayerNorm(num_embd) # Layer normalization
        self.lm_head = nn.Linear(num_embd, vocab_size) # Language modeling head, i.e. the weights to the logits

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None): 
        B, T = index.shape
        # Feed the sequence through the transformer blocks
        token_embd = self.token_embedding_table(index)
        position_embd = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = token_embd + position_embd # (B, T, C)
        x = self.blocks(x) # (B, T, C)
        x = self.ln_f(x) # (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape # batch_size, time, vocab_size
            logits = logits.view(B*T, C) # Flatten the batch and time dimensions
            targets = targets.view(B*T) # Flatten the batch and time dimensions 
            loss = F.cross_entropy(logits, targets) # Calculate the loss 

        return logits, loss

    # Generate new content
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            index_cond = index[:, -block_size:] # (B, T)
            # Make predictions
            logits, loss = self.forward(index_cond)
            # Get the last time step
            logits = logits[:, -1, :] # (B, C)
            # Turn into probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # Sample from the distribution or take the most likely
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # Append to the sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

In [63]:
model_0 = GPTModel(vocab_size).to(device)

In [ ]:
# Log to wandb
run = wandb.init(
          project = "nanoGPT",
          config = {
               "learning_rate": lr,
               "dataset": "selma-lagerlof-greatest-hits-2k24",
               "epochs": max_iterations,
               "number_of_heads": num_head,
               "number_of_layers": num_layers,
               "block_size": block_size,
               "batch_size": batch_size,
               "dropout": dropout,
               "embedding_size": num_embd,
               "split_factor": split_factor,
               "no_parameters": sum(p.numel() for p in model_0.parameters() if p.requires_grad)
          }
)

text_table = wandb.Table(columns=["epoch", "val/loss", "text"])

In [ ]:
optimizer = torch.optim.AdamW(model_0.parameters(), lr=lr)

for iter in tqdm(range(max_iterations)):
    if iter % eval_iterations == 0:
        losses = estimate_loss(model_0)
        
        # Log train and val loss to wandb
        run.log({"val/loss": losses["val"]}, step=iter)
        
        # Log text prompt "Det" to wandb
        context = torch.tensor(encode("Det"), dtype=torch.long, device=device).unsqueeze(0)
        text_table.add_data(iter, losses["val"], decode(model_0.generate(context, max_new_tokens=50)[0].tolist()))
        
        print(f'Step: {iter} | Train Loss: {losses["train"]:.4f} | Val Loss: {losses["val"]:.4f}')
    # Sample a batch of data
    xb, yb = get_batch("train")

    # Calculate the loss 
    logits, loss = model_0(xb, yb)
    # Log the loss to wandb
    run.log({"train/loss": loss.item()}, step=iter)
    optimizer.zero_grad(set_to_none=True) # Clear the gradients
    loss.backward() # Calculate the gradients
    optimizer.step() # Update the weights

print(loss.item())
run.log({"training_samples" : text_table})
run.finish()

In [ ]:
prompt = "Tjena Nisse! Hur är läget?"
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
# context = torch.zeros((1,1), dtype=torch.long, device=device)
# generated_chars = decode(model_0.generate(context, max_new_tokens=100)[0].tolist())
# print(generated_chars)